In [ ]:
# cm = confusion_matrix(y_valid, pred_valid)
# plt.figure(figsize=(10,6))
# sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
# plt.title("Confusion Matrix")
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# # plt.show()
# report = classification_report(y_valid, pred_valid, output_dict=True)
# report_df = pd.DataFrame(report).T

# plt.figure(figsize=(12,6))
# sns.heatmap(report_df.iloc[:-1, :-1], annot=True, cmap="Greens")
# plt.title("Classification Report Heatmap")
# plt.show()
# # ==========================================
# # 🔵 MULTICLASS ROC CURVE (One-vs-Rest)
# # ==========================================

# y_valid_bin = label_binarize(y_valid, classes=best_model.classes_)
# valid_probs = best_model.predict_proba(X_valid)

# plt.figure(figsize=(10,7))

# for i, cls in enumerate(best_model.classes_):
#     fpr, tpr, _ = roc_curve(y_valid_bin[:, i], valid_probs[:, i])
#     roc_auc = auc(fpr, tpr)
#     plt.plot(fpr, tpr, label=f"{cls} (AUC = {roc_auc:.2f})")

# plt.plot([0,1], [0,1], 'k--')  # diagonal line
# plt.title("Multiclass ROC Curve")
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.legend()
# plt.show()


# print("Validation Accuracy:", accuracy_score(y_valid, pred_valid))
# print("Macro F1:", f1_score(y_valid, pred_valid, average='macro'))
# print(classification_report(y_valid, pred_valid)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
df1=pd.read_csv('/kaggle/input/mse-2-ai-201-b-aiml-a/train.csv')
df2=pd.read_csv('/kaggle/input/mse-2-ai-201-b-aiml-a/test.csv')
df1['Class'].unique()
df1.isnull().sum()
print("\nChecking null values...\n")
nulls = df1.isnull().sum()

print(nulls)

plt.figure(figsize=(10,4))
sns.barplot(x=nulls.index, y=nulls.values)
plt.xticks(rotation=90)
plt.title("Null Values in Each Column")
plt.show()
# 🔍 EDA PART 2: NUMERICAL COLUMN DISTRIBUTIONS
# ==========================================
print("\nPlotting numeric distributions...\n")
X.hist(bins=30, figsize=(12,10))
plt.tight_layout()
plt.show()
print("\nPlotting category-wise countplots...\n")

plt.figure(figsize=(8, 4))
sns.countplot(x='Class', data=df1)
plt.xticks(rotation=45)
plt.title("Countplot of Class")
plt.show()
# num_cols = df1.select_dtypes(include=[np.number]).columns.tolist()
# cat_cols = df1.select_dtypes(exclude=[np.number]).columns.tolist()
# # 🔍 EDA PART 2: NUMERICAL COLUMN DISTRIBUTIONS
# # ==========================================
# print("\nPlotting numeric distributions...\n")
# X[num_cols].hist(bins=30, figsize=(12,10))
# plt.tight_layout()
# plt.show()
# # 🔍 EDA PART 3: CATEGORY-WISE DISTRIBUTION (COUNT PLOTS)
# # ==========================================
# print("\nPlotting category-wise countplots...\n")
# for col in cat_cols:
#     plt.figure(figsize=(8,4))
#     sns.countplot(data=df1, x=col)
#     plt.xticks(rotation=45)
#     plt.title(f"Countplot of {col}")
#     plt.show()
# print("\nPlotting numeric outliers (boxplots)...\n")
# for col in num_cols:
#     plt.figure(figsize=(6,4))
#     sns.boxplot(x=X[col])
#     plt.title(f"Outlier Boxplot for {col}")
#     plt.show()
y=df1["Class"]
y=y.map({"A":0,"B":1})
y=y.fillna(y.median())
X = df1.drop(["Class"],axis = 1)
X.head()
X = X.fillna(X.mean(numeric_only = True))
X.info()
X.isnull().sum()
def cap_outliers(df,cols,l=1,u = 99):
    for col in cols:
        lv = df[col].quantile(l/100)
        uv = df[col].quantile(u/100)
        df[col] = df[col].clip(lv,uv)
    return df
num_cols = X.select_dtypes(include = ["float64"]).columns
X = cap_outliers(X,num_cols)
cat_col = "Class"
for col in num_cols:
    plt.figure(figsize = (6,4))
    sns.boxplot(x = cat_col,y = col,data = df1)
sns.heatmap(X.corr())
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42,stratify = y)
X_train.head()
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier(
    random_state=42,
    class_weight="balanced"
)

param_dist = {
    "n_estimators": [200, 400, 600, 800],
    "max_depth": [None, 10, 20, 25],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

random_cv = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=25,
    cv=cv,
    scoring="f1_macro",
    random_state=42,
    verbose=1,
    n_jobs=-1
)

print("\nTraining RandomForest...")
random_cv.fit(X_train, y_train)

best_model = random_cv.best_estimator_
print("\nBest Parameters:", random_cv.best_params_)
print("="*70)
y_p = best_model.predict(X_test)
a = accuracy_score(y_test,y_p)
a
test_ids = df2["id"]
x_test = df2.drop(["id"],axis = 1)
y_pred_f = best_model.predict(x_test)
y_pred_df = pd.DataFrame(y_pred_f)
y_pred_df
y_pred1 = y_pred_df[0].map({0.0:"A",1.0:"B"})
y_pred1
submission = pd.DataFrame({
    "id" : test_ids,
    "Class" : y_pred1
})
submission.to_csv("aaaaaaaaaaaaaaaaaa.csv",index = False)
submission.head()



